In [8]:
import pandas as pd
df = pd.read_csv('survey_results_public.csv', low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98855 entries, 0 to 98854
Columns: 129 entries, Respondent to SurveyEasy
dtypes: float64(41), int64(1), object(87)
memory usage: 97.3+ MB


In [11]:
import numpy as np
category_columns = [
    column for column in df.columns if df[column].nunique() < 100 and df[column].dtype == np.dtype('O')
]
print(len(category_columns))

69


In [12]:
for column in category_columns:
    df[column] = df[column].astype('category')

In [17]:
import json
data_dtypes = {column:str(df[column].dtype) for column in df.columns}
json.dump(data_dtypes, open('data_dtypes.json', 'w'))

In [ ]:
df = pd.read_csv('survey_results_public.csv', low_memory=False)
df.info()